**Please run the cells marked ### RUN ### sequentially to get the final output**
### Some points to consider before working on this project(These points are not necessary if the zip file is downloaded):
* All of this work has been done on Google Colab. So, please run it on Google Colab.
* Firstly clone the UNeXt repo and inside that directory create a folder namely "inputs". Inside "inputs" folder keep your dataset folder say "BUSI". Inside "BUSI" make two folders "images" and "masks".
* Inside "images" store the images and inside "masks" folder create another directory say "0","1","2" the number of classes you have inside the dataset. 
* Inside each class store the respective class segmented maps with the same filename as the image in the "images" folder.(Do not use the double quotes while naming they have been used just for highlighting purposes).

In [ ]:
!git clone https://github.com/jeya-maria-jose/UNeXt-pytorch
%cd UNeXt-pytorch
!mkdir inputs
%cd inputs
!mkdir Dataset_BUSI_with_GT
%cd Dataset_BUSI_with_GT
!mkdir images
!mkdir masks
%cd masks
!mkdir 0
%cd ../..
!mkdir ISIC2018_Task1-2_Training_Input
%cd ISIC2018_Task1-2_Training_Input
!mkdir images
!mkdir masks
%cd masks
!mkdir 0
%cd ../../..
!pwd

# This cell clones the UNeXt repo and downloads the data from **Google Drive**

In [ ]:
### RUN ###
# https://drive.google.com/file/d/1PgegjqOwaPAII3HfUaZyUFX6K8xVe5EG/view?usp=sharing
!git clone https://github.com/jeya-maria-jose/UNeXt-pytorch
%cd UNeXt-pytorch
!pip install gdown -q
!gdown --id 1PgegjqOwaPAII3HfUaZyUFX6K8xVe5EG
!unzip -q inputs.zip
!rm -rf inputs.zip
!ls

Cloning into 'UNeXt-pytorch'...
remote: Enumerating objects: 59, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 59 (delta 20), reused 26 (delta 5), pack-reused 0
Unpacking objects: 100% (59/59), done.
/content/UNeXt-pytorch
/usr/local/lib/python3.7/dist-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1PgegjqOwaPAII3HfUaZyUFX6K8xVe5EG
To: /content/UNeXt-pytorch/inputs.zip
100% 875M/875M [00:10<00:00, 86.1MB/s]
archs.py    environment.yml  LICENSE	 post_process.py  train.py
config.py   imgs	     losses.py	 __pycache__	  utils.py
dataset.py  inputs	     metrics.py  README.md	  val.py


## **This cell is for downloading the token from kaggle.json**


In [ ]:
#plant datset making connection to kaggle to download the dataset
from google.colab import files
files.upload()
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 kaggle.json

## **This cell downloads the BUSI dataset from kaggle**

In [ ]:
#plant dataset extracting the data using api from kaggle
!kaggle datasets download -d aryashah2k/breast-ultrasound-images-dataset
!ls

# **Unzipping the file**

In [ ]:
#unzipping plant dataset 
!unzip -q breast-ultrasound-images-dataset.zip
!rm -rf breast-ultrasound-images-dataset.zip
!ls

In [ ]:
import wget
url = "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1-2_Training_Input.zip"
wget.download(url, './')

url = "https://isic-challenge-data.s3.amazonaws.com/2018/ISIC2018_Task1_Training_GroundTruth.zip"
wget.download(url, './')

!unzip -q ISIC2018_Task1-2_Training_Input.zip
!rm -rf ISIC2018_Task1-2_Training_Input.zip
!unzip -q ISIC2018_Task1_Training_GroundTruth.zip
!rm -rf ISIC2018_Task1_Training_GroundTruth.zip
!ls

# **Libraries required to run the code**

In [ ]:
### RUN ###
!pip install timm==0.3.2
!pip install mmcv
!pip install torch==1.7.1
!pip install torchvision==0.8.2
!pip install -U albumentations
!pip uninstall opencv-python
!pip install opencv-python==4.5.1.48
!pip install --ignore-installed PyYAML

     |████████████████████████████████| 244 kB 26.8 MB/s 
     |████████████████████████████████| 530 kB 29.5 MB/s 
     |████████████████████████████████| 190 kB 36.6 MB/s 
  Created wheel for mmcv: filename=mmcv-1.5.0-py2.py3-none-any.whl size=807180 sha256=2667e88a017b6fe64ee8275f1d96f8ced030cb9bb0dd93268798fa178d46ec57
  Stored in directory: /root/.cache/pip/wheels/09/b4/88/0fee9e050a776e49b7290c2853e8ec002eeb55d7f4a40cf78f
Successfully built mmcv
     |████████████████████████████████| 776.8 MB 17 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.11.0+cu113
    Uninstalling torch-1.11.0+cu113:
      Successfully uninstalled torch-1.11.0+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.12.0+cu113 requires torch==1.11.0, but you have torch 1.7.1 which is incompatible.
torchtext 0.12.0 requires torch==1.11.0, but 

## **Code for creating the required directory structure for ISIC Dataset**

In [ ]:
# For ISIC 2018 dataset
from pathlib import Path
import shutil
import os
import glob
import re

src_path_images ="./ISIC2018_Task1-2_Training_Input/"
src_path_masks = "./ISIC2018_Task1_Training_GroundTruth/"
trg_path_images = "./inputs/ISIC2018_Task1-2_Training_Input/images/"
trg_path_masks = "./inputs/ISIC2018_Task1-2_Training_Input/masks/"

files = os.listdir(src_path_images)
print(type(files))
for f in files:
    regex = r".jpg$"
    if re.search(regex, f) != None: 
        shutil.move(os.path.join(src_path_images,f), trg_path_images)

files = os.listdir(src_path_masks)
for f in files:
    regex = r".png$"
    if re.search(regex, f) != None: 
        shutil.move(os.path.join(src_path_masks,f), trg_path_masks+"0/"+f.replace('_segmentation',''))


## **Code for creating the directory structure for BUSI Dataset**

In [ ]:
# For BUSI dataset
from pathlib import Path
import shutil
import os
import glob
import re

src_path_benign ="./Dataset_BUSI_with_GT/benign/"
src_path_malignant = "./Dataset_BUSI_with_GT/malignant/"
src_path_normal = "./Dataset_BUSI_with_GT/normal/"
trg_path_inputs = "./inputs/Dataset_BUSI_with_GT/images/"
trg_path_masks = "./inputs/Dataset_BUSI_with_GT/masks/"

files = os.listdir(src_path_benign)
print(type(files))
for f in files:
    regex = r"\)_mask.png$"
    regex1 = r"\).png$"
    if re.search(regex, f) != None:
        shutil.move(os.path.join(src_path_benign,f), trg_path_masks+"0/"+f.replace('_mask',''))
    elif re.search(regex1, f) != None:
        shutil.move(os.path.join(src_path_benign,f), trg_path_inputs)

files = os.listdir(src_path_malignant)
for f in files:
    regex = r"\)_mask.png$"
    regex1 = r"\).png$"
    if re.search(regex, f) != None:
        shutil.move(os.path.join(src_path_malignant,f), trg_path_masks+"0/"+f.replace('_mask',''))
    elif re.search(regex1, f) != None:
        shutil.move(os.path.join(src_path_malignant,f), trg_path_inputs)

files = os.listdir(src_path_normal)
for f in files:
    regex = r"\)_mask.png$"
    regex1 = r"\).png$"
    if re.search(regex, f) != None or re.search(regex1, f) != None:
        shutil.move(os.path.join(src_path_normal,f), trg_path_masks+"0/"+f.replace('_mask',''))
    elif re.search(regex1, f) != None:
        shutil.move(os.path.join(src_path_normal,f), trg_path_inputs)

<class 'list'>


## **Command to train the model with the specified parameters like model name, dataset name, etc.**

In [ ]:
### RUN ###
# Run this command for BUSI dataset
!python train.py --dataset Dataset_BUSI_with_GT --arch UNext --name Dataset_BUSI_with_GT --img_ext .png --mask_ext .png --lr 0.0001 --epochs 100 --input_w 256 --input_h 256 --b 8
# Run this command for ISIC dataset
#!python train.py --dataset ISIC2018_Task1-2_Training_Input --arch UNext --name ISIC2018_Task1-2_Training_Input --img_ext .jpg --mask_ext .png --lr 0.0001 --epochs 100 --input_w 256 --input_h 256 --b 8

## **Command to generating the predictions over the test data**

In [ ]:
### RUN ###
# Run this command for validating BUSI dataset
!python val.py --name Dataset_BUSI_with_GT
# Run this command for validation ISIC dataset
#!python val.py --name ISIC2018_Task1-2_Training_Input

In [ ]:
# Check if GPU is enabled.
!nvidia-smi
!lscpu |grep 'Model name'

Thu Apr 28 00:21:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# This code snippet will read the images from the output folder and from original segmented **folder**

In [ ]:
### RUN ###
import cv2
import os
import numpy as np
predmaskarr=[]
origmaskarr=[]
outpath ='/content/UNeXt-pytorch/outputs/ISIC2018_Task1-2_Training_Input/0/'
#inpath = '/content/UNeXt-pytorch/inputs/Dataset_BUSI_with_GT/masks/0/'
inpath = '/content/UNeXt-pytorch/inputs/ISIC2018_Task1-2_Training_Input/masks/0/'
for img1 in os.listdir(outpath):
  im1=cv2.imread(outpath+img1)
  predmaskarr.append(np.array(im1))
  for img2 in os.listdir(inpath):
    #print(img1,img2)
    if(img1[:-3] == img2[:-3]):
      im2=cv2.imread(inpath+img2)
      imresized = cv2.resize(im2, (256,256), interpolation = cv2.INTER_NEAREST)
      #print(np.array(imresized).shape)
      origmaskarr.append(np.array(imresized))


In [ ]:
### RUN ###
print(np.array(predmaskarr).shape)
print(np.array(origmaskarr).shape)

(519, 256, 256, 3)
(519, 256, 256, 3)


# Computes the F1-score **here**

In [ ]:
### RUN ###
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

f1=0

for i in range(0,len(predmaskarr)):
  f1= f1+f1_score(origmaskarr[i].flatten(),predmaskarr[i].flatten(),average='micro')
f1=f1/len(predmaskarr)
print(f1)
'''
print("Precision Score : ",precision_score(origmaskarr,predmaskarr,pos_label='positive'))
print("Recall Score :" , recall_score(origmaskarr,predmaskarr, pos_label='positive') )
'''

0.942051176390896


'\nprint("Precision Score : ",precision_score(origmaskarr,predmaskarr,pos_label=\'positive\'))\nprint("Recall Score :" , recall_score(origmaskarr,predmaskarr, pos_label=\'positive\') )\n'